## Evaluating with the Azure AI Evaluation SDK

https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/flow-evaluate-sdk

### An Azure OpenAI resource created

To evaluate with AI-assisted metrics, you need:

A test dataset in .jsonl format. See the next section for dataset requirements
A deployment of one of these models: GPT 3.5 models, GPT 4 models, or Davinci models AND an embedding model for grounded responses with RAG.
Ideally, GPT 4 models are recommended for the best evaluation capabilities.


## Add steps to create an Azure OpenAI resource and deploy a model

### Install SDK

In [ ]:
# install into current notebook environment
import sys
!{sys.executable} -m pip install azure-ai-evaluation

### Prepare config files

#### create .env file containing secrets
```
SUBSCRIPTION_ID=
RESOURCE_GROUP_NAME=
PROJECT_NAME=
AZURE_OPENAI_ENDPOINT=
AZURE_OPENAI_EVALUATION_DEPLOYMENT=
# Uncomment if using key-based auth
# AZURE_OPENAI_KEY=
```

In [1]:
from dotenv import load_dotenv
load_dotenv('../.env', override=True)

True

### Initialise Azure OpenAI connection

#### Make sure user is Azure OpenAI contributor
https://learn.microsoft.com/en-us/azure/ai-studio/concepts/rbac-ai-studio#scenario-use-an-existing-azure-openai-resource

In [ ]:
import os

# Initialize Azure OpenAI Connection with your environment variables
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

In [31]:
import os
from promptflow.core import AzureOpenAIModelConfiguration

# Initialize Azure OpenAI Connection with your environment variables
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    azure_deployment=os.environ.get("AZURE_OPENAI_EVALUATION_DEPLOYMENT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

# #Initialize Azure OpenAI Connection with your environment variables
# model_config = AzureOpenAIModelConfiguration(
#     azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
#     azure_deployment=os.environ.get("AZURE_OPENAI_EVALUATION_DEPLOYMENT"),
#     api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
# )

# Initialize Azure OpenAI Connection using AI Studio connection
# from promptflow.rag.config import ConnectionConfig

# model_connect_config = ConnectionConfig(
#     subscription_id = os.environ.get("SUBSCRIPTION_ID"),
#     resource_group_name = os.environ.get("RESOURCE_GROUP_NAME"),
#     workspace_name = os.environ.get("PROJECT_NAME"),
#     connection_name = "mssecureai4034688619"
# )
# model_connect = AzureOpenAIConnection(
#     api_base = os.environ.get("AZURE_OPENAI_ENDPOINT"),
#     auth_mode = "meid_token"
# )
model_connect = AzureOpenAIConnection(
    name=os.environ.get("AISTUDIO_AOAI_CONNECTION_NAME"),
    api_base=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_type="azure",
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

# model_config = AzureOpenAIModelConfiguration(
#     azure_deployment="gpt-4o",
#     connection=model_connect
# )

In [2]:
# # try using connection config
# # from promptflow.rag.config import ConnectionConfig

# model_connect_config = ConnectionConfig(
#     subscription_id = os.environ.get("SUBSCRIPTION_ID"),
#     resource_group_name = os.environ.get("RESOURCE_GROUP_NAME"),
#     workspace_name = os.environ.get("PROJECT_NAME"),
#     connection_name = "mssecureai4034688619"

# model_connect = AzureOpenAIModelConfiguration.from_connection(model_connect_config)

In [32]:
from promptflow.evals.evaluators import RelevanceEvaluator

# Initialzing Relevance Evaluator
relevance_eval = RelevanceEvaluator(model_config)
# Running Relevance Evaluator on single input row
relevance_score = relevance_eval(
    answer="The Alpine Explorer Tent is the most waterproof.",
    context="From the our product list,"
    " the alpine explorer tent is the most waterproof."
    " The Adventure Dining Table has higher weight.",
    question="Which tent is the most waterproof?",
)
print(relevance_score)

{'gpt_relevance': 5.0}


#### Test Risk and Safety Evaluators
GPT not required - instead we use Azure AI Studio safety evaluations back-end service.

Note - Risk and safety metrics are only available in the following regions: East US 2, France Central, UK South, Sweden Central. 

***Groundedness measurement leveraging Azure AI Content Safety Groundedness Detection is only supported following regions: East US 2 and Sweden Central.***

Check [region-availability](https://learn.microsoft.com/en-us/azure/ai-services/content-safety/overview#region-availability)

In [5]:
# from azure.identity import DefaultAzureCredential, get_bearer_token_provider
# token_provider = get_bearer_token_provider(
#     DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
# )

In [33]:
# define the Azure AI Studio connection
azure_ai_project = {
    "subscription_id": os.environ.get("SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("RESOURCE_GROUP_NAME"),
    "project_name": os.environ.get("PROJECT_NAME"),
    #"credential": token_provider,
}


#### Test it out

In [34]:
from promptflow.evals.evaluators import ViolenceEvaluator

# Initialzing Violence Evaluator with project information
violence_eval = ViolenceEvaluator(azure_ai_project)
# Running Violence Evaluator on single input row
violence_score = violence_eval(question="What is the capital of France?", answer="Paris.")
print(violence_score)

[2024-08-05 21:14:59 +1000][flowinvoker][INFO] - Getting connections from pf client with provider from args: local...
[2024-08-05 21:14:59 +1000][flowinvoker][INFO] - Promptflow get connections successfully. keys: dict_keys([])
[2024-08-05 21:14:59 +1000][flowinvoker][INFO] - Promptflow executor starts initializing...
[2024-08-05 21:14:59 +1000][flowinvoker][INFO] - Promptflow executor initiated successfully.
[2024-08-05 21:14:59 +1000][flowinvoker][INFO] - Validating flow input with data {'metric_name': 'violence', 'question': 'What is the capital of France?', 'answer': 'Paris.', 'project_scope': {'subscription_id': '3c8972d9-f541-46b2-b70b-d81baba3595d', 'resource_group_name': 'secure-ai-rg', 'project_name': 'krbock-0635'}, 'credential': None}
[2024-08-05 21:14:59 +1000][flowinvoker][INFO] - Execute flow with data {'metric_name': 'violence', 'question': 'What is the capital of France?', 'answer': 'Paris.', 'project_scope': {'subscription_id': '3c8972d9-f541-46b2-b70b-d81baba3595d', '

2024-08-05 21:14:59 +1000     816 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-08-05 21:14:59 +1000     816 execution.flow     INFO     Start to run 2 nodes with concurrency level 16.
2024-08-05 21:14:59 +1000     816 execution.flow     INFO     Executing node validate_inputs. node run id: d1f52e2a-beaf-4524-a04b-2c0b71361418_validate_inputs_67a661ef-f66d-4205-95c3-0b75dc58f374
2024-08-05 21:14:59 +1000     816 execution.flow     INFO     Node validate_inputs completes.
2024-08-05 21:14:59 +1000     816 execution.flow     INFO     The node 'evaluate_with_rai_service' will be executed because the activate condition is met, i.e. '${validate_inputs.output}' is equal to 'True'.
2024-08-05 21:14:59 +1000     816 execution.flow     INFO     Executing node evaluate_with_rai_service. node run id: d1f52e2a-beaf-4524-a04b-2c0b71361418_evaluate_with_rai_service_93a9082c-83fc-40f3-bd36-e5f721e3cb9e
2024-08-05 21:15:09 +1000     816 execution.flow     INFO     Node ev

### Examine local dataset

In [35]:
import json
def load_jsonl(path):
    with open(path, "r") as f:
        return [json.loads(line) for line in f.readlines()]
    
mydata = load_jsonl('../data/evaluation_dataset.jsonl')
mydata[:10]

[{'question': 'Which tent is the most waterproof?',
  'truth': 'The Alpine Explorer Tent has the highest rainfly waterproof rating at 3000m'},
 {'question': 'Which camping table holds the most weight?',
  'truth': 'The Adventure Dining Table has a higher weight capacity than all of the other camping tables mentioned'},
 {'question': 'How much does TrailWalker Hiking Shoes cost? ',
  'truth': '$110'},
 {'question': 'What is the proper care for trailwalker hiking shoes? ',
  'truth': 'After each use, remove any dirt or debris by brushing or wiping the shoes with a damp cloth.'},
 {'question': 'What brand is for TrailMaster tent? ',
  'truth': 'OutdoorLiving'},
 {'question': 'How do I carry the TrailMaster tent around? ',
  'truth': ' Carry bag included for convenient storage and transportation'},
 {'question': 'What is the floor area for Floor Area? ',
  'truth': '80 square feet'},
 {'question': 'What is the material for TrailBlaze Hiking Pants',
  'truth': 'Made of high-quality nylon fa

In [36]:
import os
# create directory for output
output_dir = '../data/evaluate'
os.makedirs(output_dir, exist_ok=True)

In [37]:
# callable function that invokes Azure OpenAI.  For use as target in evaluator.
from genai.llm import llm_tool

| Evaluator | question | answer | context | ground_truth |
| --- | --- | --- | --- | --- | 
| GroundednessEvaluator | N/A | Required: String | Required: String | N/A |
| RelevanceEvaluator | Required: String | Required: String | Required: String | N/A |
| CoherenceEvaluator | Required: String | Required: String | N/A | N/A |
| FluencyEvaluator | Required: String | Required: String | N/A | N/A |
| SimilarityEvaluator | Required: String | Required: String | N/A | Required: String |
| F1ScoreEvaluator | N/A | Required: String | N/A | Required: String |
| ViolenceEvaluator | Required: String | Required: String | N/A | N/A |
| SexualEvaluator | Required: String | Required: String | N/A | N/A |
| SelfHarmEvaluator | Required: String | Required: String | N/A | N/A |
| HateUnfairnessEvaluator | Required: String | Required: String | N/A | N/A |

### Run a  qa evaluation against the AI studio to ensure the connection is working

In [38]:
from promptflow.evals.evaluators import CoherenceEvaluator, RelevanceEvaluator, GroundednessEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator

coherence_eval = CoherenceEvaluator(model_config=model_config)
relevance_eval = RelevanceEvaluator(model_config=model_config)
groundedness_eval = GroundednessEvaluator(model_config=model_config)
fluency_eval = FluencyEvaluator(model_config=model_config)
similarity_eval = SimilarityEvaluator(model_config=model_config)
f1score_eval = F1ScoreEvaluator()


In [10]:
from promptflow.evals.evaluate import evaluate

result = evaluate(
    evaluation_name="rai-workshop-test", #name your evaluation to view in AI Studio
    data='../data/evaluation_dataset.jsonl', # provide your data here - must be string
    target=llm_tool,
    evaluators={
        #"relevance": relevance_eval,
        "coherence": coherence_eval,
        #"groundedness": groundedness_eval,
        "fluency": fluency_eval,
        "similarity": similarity_eval,
        "f1score": f1score_eval

    },
    # column mapping
    evaluator_config={
        "default": {
            "questions": "${data.question)", #column of data providing input to model
            #"contexts": "${data.context}", #column of data providing context for each input
            "answer": "${target.answer}", #column of data providing output from model
            "ground_truth":"${data.truth}" #column of data providing ground truth answer, optional for default metrics
        }
    },
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI studio project
    azure_ai_project = azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and studio URL
    output_path=output_dir
)

[2024-08-02 15:46:12 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Upload run to cloud: True


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=n0_prerequisites_20240802_154612_455515
You can view the traces in azure portal since trace destination is set to: azureml://subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635. The link will be printed once the run is finished.


[2024-08-02 15:46:16 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run n0_prerequisites_20240802_154612_455515, log path: /home/krbock/.promptflow/.runs/n0_prerequisites_20240802_154612_455515/logs.txt


2024-08-02 15:46:39 +1000   30491 execution.bulk     INFO     Process 30521 terminated.
2024-08-02 15:46:39 +1000   30491 execution.bulk     INFO     Process 30533 terminated.
2024-08-02 15:46:39 +1000   30491 execution.bulk     WARNING  Process 30526 had been terminated.
2024-08-02 15:46:39 +1000   30491 execution.bulk     WARNING  Process 30516 had been terminated.


[2024-08-02 15:46:40 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Uploading run 'n0_prerequisites_20240802_154612_455515' to cloud...
[2024-08-02 15:47:00 +1000][promptflow._sdk._orchestrator.run_submitter][INFO] - Updating run 'n0_prerequisites_20240802_154612_455515' portal url to 'https://ai.azure.com/projectflows/trace/run/n0_prerequisites_20240802_154612_455515/details?wsid=/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourcegroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635'.


...

2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Start to run 2 nodes with concurrency level 16.
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Current thread is not main thread, skip signal handler registration in AsyncNodesScheduler.
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Executing node validate_inputs. node run id: 49349024-b5a3-4fdd-8dfd-2ab2a36e3234_validate_inputs_822f0801-d089-43dd-a6ff-69caeb4e1f9f
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Node validate_inputs completes.


[2024-08-02 15:47:06 +1000][flowinvoker][INFO] - Validating flow input with data {'answer': "The PowerBurner Camping Stove is typically made from a combination of durable materials designed to withstand high temperatures and outdoor conditions. Common materials include stainless steel for the burner and body, aluminum for lightweight components, and heat-resistant plastics for handles and knobs. These materials ensure the stove is both robust and portable, making it ideal for camping and outdoor use. Always check the specific model's specifications for precise material details.", 'ground_truth': 'Stainless Steel'}
[2024-08-02 15:47:06 +1000][flowinvoker][INFO] - Execute flow with data {'answer': "The PowerBurner Camping Stove is typically made from a combination of durable materials designed to withstand high temperatures and outdoor conditions. Common materials include stainless steel for the burner and body, aluminum for lightweight components, and heat-resistant plastics for handles

2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     The node 'compute_f1_score' will be executed because the activate condition is met, i.e. '${validate_inputs.output}' is equal to 'True'.
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Start executing nodes in thread pool mode.


[2024-08-02 15:47:06 +1000][flowinvoker][INFO] - Validating flow input with data {'answer': "Yes, that's correct! France is a country located in Western Europe. It's known for its rich history, culture, cuisine, and landmarks such as the Eiffel Tower and the Louvre Museum. Is there something specific you'd like to know about France?", 'ground_truth': 'Sorry, I can only truth questions related to outdoor/camping gear and equipment'}
[2024-08-02 15:47:06 +1000][flowinvoker][INFO] - Execute flow with data {'answer': "Yes, that's correct! France is a country located in Western Europe. It's known for its rich history, culture, cuisine, and landmarks such as the Eiffel Tower and the Louvre Museum. Is there something specific you'd like to know about France?", 'ground_truth': 'Sorry, I can only truth questions related to outdoor/camping gear and equipment'}


2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Executing node compute_f1_score. node run id: 49349024-b5a3-4fdd-8dfd-2ab2a36e3234_compute_f1_score_b2515c57-5ee0-467f-a674-83abf9d7f546
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Start to run 2 nodes with concurrency level 16.
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Node compute_f1_score completes.
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Current thread is not main thread, skip signal handler registration in AsyncNodesScheduler.
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Start to run 2 nodes with concurrency level 16.
2024-08-02 15:47:06 +1000   29780 execution.flow     INFO     Executing node validate_inputs. node run id: 59d474ea-8fd2-4e51-a65f-8ca367a0019e_validate_inputs_07d71376-6257-4278-8a38-ea2720fc0a4e
2024-08-02 15:47:06 +1000   29

### Create a Retrieval Augmented Generation (RAG) application using Promptflow SDK

We will use the RAG pattern to validate our model against ground-truth.

#### Data
This sample uses files from the folder data/ in this repo. You can clone this repo or copy this folder to make sure you have access to these files when running the sample.

In [12]:
# from earlier step
# import json
# def load_jsonl(path):
#     with open(path, "r") as f:
#         return [json.loads(line) for line in f.readlines()]

#mydata = load_jsonl('../data/evaluation_dataset.jsonl')

mydata[:2]

[{'question': 'Which tent is the most waterproof?',
  'truth': 'The Alpine Explorer Tent has the highest rainfly waterproof rating at 3000m'},
 {'question': 'Which camping table holds the most weight?',
  'truth': 'The Adventure Dining Table has a higher weight capacity than all of the other camping tables mentioned'}]

#### Create an local FAISS index from your local files
https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/index-build-consume-sdk

Install the FAISS package with
```
"Please install it with `pip install faiss-gpu` (for CUDA supported GPU) "
    "or `pip install faiss-cpu` (depending on Python version)."
```

In [11]:
# connect to the AI Studio project
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

client=MLClient(
    DefaultAzureCredential(), 
    subscription_id=os.environ.get("SUBSCRIPTION_ID"),
    resource_group_name=os.environ.get("RESOURCE_GROUP_NAME"),
    workspace_name=os.environ.get("PROJECT_NAME") 
)

#### Use AIStudio's Azure OpenAI connection

In [14]:
from promptflow.rag.config import ConnectionConfig
# embedding_model_config = ConnectionConfig(
#     subscription_id = os.environ.get("SUBSCRIPTION_ID"),
#     resource_group_name = os.environ.get("RESOURCE_GROUP_NAME"),
#     workspace_name = os.environ.get("PROJECT_NAME"),
#     connection_name = "Default_AzureOpenAI"
# )

embedding_model_config = ConnectionConfig(
    subscription_id = os.environ.get("SUBSCRIPTION_ID"),
    resource_group_name = os.environ.get("RESOURCE_GROUP_NAME"),
    workspace_name = os.environ.get("PROJECT_NAME"),
    connection_name = os.environ.get("AISTUDIO_AOAI_CONNECTION_NAME"),
)

In [15]:
from promptflow.rag.config import LocalSource, EmbeddingsModelConfig
from promptflow.rag import build_index

faiss_index_name = "product-info-faiss-index"
embedding_output_dir = "../data"

# build the index
faiss_index=build_index(
    name=faiss_index_name,  # name of your index
    vector_store="faiss",  # the type of vector store
    embeddings_model_config=EmbeddingsModelConfig(
        model_name=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
        deployment_name=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
        connection_config=embedding_model_config
    ),
    input_source=LocalSource(input_data="../data/product-info/"),  # the location of your file/folders
    #index_config=LocalSource(input_data="../data/product-info/"
        #ai_search_index_name="<your-index-name>" + "-aoai-store", # the name of the index store inside the azure ai search service
    #),
    tokens_per_chunk = 800, # Optional field - Maximum number of tokens per chunk
    token_overlap_across_chunks = 0, # Optional field - Number of tokens to overlap between chunks
    embeddings_cache_path=embedding_output_dir, # Optional field - Path to store embeddings cache
)

Crack and chunk files from local path: ../data/product-info/


INFO:azureml.rag.connections:Using ml_client base_url: https://management.azure.com, original_base_url: https://management.azure.com.
INFO:azureml.rag.connections:Parsed Connection: /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureai4034688619_aoai
INFO:azureml.rag.connections:Got connection: /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureai4034688619_aoai as <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureOpenAIConnection'>.
INFO:azureml.rag.connections:Getting workspace connection: mssecureai4034688619_aoai, with input credential: <class 'NoneType'>.
INFO:azureml.rag.connections:Getting workspace connection via MLClient with auth: <class 'azure.identity._credentials.default.DefaultAzureCredential'>, subscription_id: 3c897

Start embedding using connection with id = /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureai4034688619_aoai


INFO:azureml.rag.connections:Parsed Connection: /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureai4034688619_aoai
INFO:azureml.rag.connections:Got connection: /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureai4034688619_aoai as <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureOpenAIConnection'>.
INFO:azureml.rag.connections:The connection 'mssecureai4034688619_aoai' is a <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureOpenAIConnection'> with api_key auth type.
INFO:azureml.rag.azureml.rag.documents:[DocumentChunksIterator::filter_extensions] Filtered 0 files out of 20
INFO:azureml.rag.azureml.rag.documents.cracking:[DocumentChunksIterator::crack_documents] Total time to load files: 0.001402854919

Successfully created index at ../data/product-info-faiss-index-mlindex


#### Consume index

In [27]:
from promptflow.rag import get_langchain_retriever_from_index

# Get the OpenAI embedded Index
#retriever=get_langchain_retriever_from_index(faiss_index)
retriever=get_langchain_retriever_from_index(faiss_index)
retriever.get_relevant_documents("Which tent is the most waterproof")


NameError: name 'faiss_index' is not defined

#### Register Index (Optional)

In [ ]:
# connect to the AI Studio project
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

client=MLClient(
    DefaultAzureCredential(), 
    subscription_id=os.environ.get("SUBSCRIPTION_ID"),
    resource_group_name=os.environ.get("RESOURCE_GROUP_NAME"),
    workspace_name=os.environ.get("PROJECT_NAME") 
)

In [ ]:
from azure.ai.ml.entities import Index

# register the index with Azure OpenAI embeddings
client.indexes.create_or_update(
    Index(name=faiss_index_name + "aoai", 
          path=faiss_index, 
          version="1")
          )

#### Option 2: Use Azure AI Search to create an index


In [39]:
from promptflow.rag.config import ConnectionConfig
embedding_model_config = ConnectionConfig(
    subscription_id = os.environ.get("SUBSCRIPTION_ID"),
    resource_group_name = os.environ.get("RESOURCE_GROUP_NAME"),
    workspace_name = os.environ.get("PROJECT_NAME"),
    connection_name = os.environ.get("AISTUDIO_AOAI_CONNECTION_NAME"),
)

ais_model_config = ConnectionConfig(
    subscription_id = os.environ.get("SUBSCRIPTION_ID"),
    resource_group_name = os.environ.get("RESOURCE_GROUP_NAME"),
    workspace_name = os.environ.get("PROJECT_NAME"),
    connection_name = os.environ.get("AISTUDIO_AIS_CONNECTION_NAME"),
)

In [40]:
# connect to the AI Studio project
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

client=MLClient(
    DefaultAzureCredential(), 
    subscription_id=os.environ.get("SUBSCRIPTION_ID"),
    resource_group_name=os.environ.get("RESOURCE_GROUP_NAME"),
    workspace_name=os.environ.get("PROJECT_NAME")
    )

In [ ]:
## from https://learn.microsoft.com/en-us/azure/ai-studio/tutorials/copilot-sdk-build-rag
import os
from dotenv import load_dotenv

load_dotenv()

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Index

from promptflow.rag.config import (
    LocalSource,
    AzureAISearchConfig,
    EmbeddingsModelConfig,
    ConnectionConfig,
)
from promptflow.rag import build_index

client = MLClient(
    DefaultAzureCredential(),
    os.getenv("AZURE_SUBSCRIPTION_ID"),
    os.getenv("AZURE_RESOURCE_GROUP"),
    os.getenv("AZUREAI_PROJECT_NAME"),
)
import os

# append directory of the current script to data directory
script_dir = os.path.dirname(os.path.abspath(__file__))
data_directory = os.path.join(script_dir, "data/product-info/")

# Check if the directory exists
if os.path.exists(data_directory):
    files = os.listdir(data_directory)  # List all files in the directory
    if files:
        print(
            f"Data directory '{data_directory}' exists and contains {len(files)} files."
        )
    else:
        print(f"Data directory '{data_directory}' exists but is empty.")
        exit()
else:
    print(f"Data directory '{data_directory}' does not exist.")
    exit()

index_name = "tutorial-index"  # your desired index name
index_path = build_index(
    name=index_name,  # name of your index
    vector_store="azure_ai_search",  # the type of vector store - in this case it is Azure AI Search. Users can also use "azure_cognitive search"
    embeddings_model_config=EmbeddingsModelConfig(
        model_name=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
        deployment_name=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
        connection_config=ConnectionConfig(
            subscription_id=client.subscription_id,
            resource_group_name=client.resource_group_name,
            workspace_name=client.workspace_name,
            connection_name=os.getenv("AZURE_OPENAI_CONNECTION_NAME"),
        ),
    ),
    input_source=LocalSource(input_data=data_directory),  # the location of your files
    index_config=AzureAISearchConfig(
        ai_search_index_name=index_name,  # the name of the index store inside the azure ai search service
        ai_search_connection_config=ConnectionConfig(
            subscription_id=client.subscription_id,
            resource_group_name=client.resource_group_name,
            workspace_name=client.workspace_name,
            connection_name=os.getenv("AZURE_SEARCH_CONNECTION_NAME"),
        ),
    ),
    tokens_per_chunk=800,  # Optional field - Maximum number of tokens per chunk
    token_overlap_across_chunks=0,  # Optional field - Number of tokens to overlap between chunks
)

# register the index so that it shows up in the cloud project
client.indexes.create_or_update(Index(name=index_name, path=index_path))

In [41]:
from promptflow.rag.config import AzureAISearchConfig, EmbeddingsModelConfig, LocalSource
from promptflow.rag import build_index

ais_index_name = "product-info-ais-index"
embedding_output_dir = "../data"

local_index_aoai=build_index(
    name=ais_index_name,  # name of your index
    vector_store="azure_ai_search",  # the type of vector store
    embeddings_model_config=EmbeddingsModelConfig(
        model_name=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
        deployment_name=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
        connection_config=embedding_model_config
    ),
    input_source=LocalSource(input_data="../data/product-info/"),  # the location of your file/folders
    index_config=AzureAISearchConfig(
        ai_search_index_name="product-info-ais-index" + "-aoai-store", # the name of the index store inside the azure ai search service
        ai_search_connection_config=ais_model_config
    ),
    tokens_per_chunk = 800, # Optional field - Maximum number of tokens per chunk
    token_overlap_across_chunks = 0, # Optional field - Number of tokens to overlap between chunks
    embeddings_cache_path=embedding_output_dir, # Optional field - Path to store embeddings cache
)

INFO:azureml.rag.connections:Getting workspace connection: mssecureai4034688619_aoai, with input credential: <class 'NoneType'>.
INFO:azureml.rag.connections:Getting workspace connection via MLClient with auth: <class 'azure.identity._credentials.default.DefaultAzureCredential'>, subscription_id: 3c8972d9-f541-46b2-b70b-d81baba3595d, resource_group_name: secure-ai-rg, workspace_name: krbock-0635.


Crack and chunk files from local path: ../data/product-info/


Method connections: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
INFO:azureml.rag.connections:Using ml_client base_url: https://management.azure.com, original_base_url: https://management.azure.com.
INFO:azureml.rag.connections:Parsed Connection: /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureai4034688619_aoai
INFO:azureml.rag.connections:Got connection: /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureai4034688619_aoai as <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureOpenAIConnection'>.
INFO:azureml.rag.connections:Getting workspace connection: mssecureai4034688619_aoai, with input credential: <class 'NoneType'>.
INFO:azureml.rag.connectio

Start embedding using connection with id = /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureai4034688619_aoai


INFO:azureml.rag.connections:Parsed Connection: /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureai4034688619_aoai
INFO:azureml.rag.connections:Got connection: /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureai4034688619_aoai as <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureOpenAIConnection'>.
INFO:azureml.rag.connections:The connection 'mssecureai4034688619_aoai' is a <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureOpenAIConnection'> with api_key auth type.
INFO:azureml.rag.azureml.rag.documents:[DocumentChunksIterator::filter_extensions] Filtered 0 files out of 20
INFO:azureml.rag.azureml.rag.documents.cracking:[DocumentChunksIterator::crack_documents] Total time to load files: 0.000646591186

Start creating index from embeddings.


INFO:azureml.rag.connections:Parsed Connection: /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureaisearch
INFO:azureml.rag.connections:Got connection: /subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/secure-ai-rg/providers/Microsoft.MachineLearningServices/workspaces/krbock-0635/connections/mssecureaisearch as <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureAISearchConnection'>.
INFO:azureml.rag.connections:The connection 'mssecureaisearch' is a <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureAISearchConnection'> with api_key auth type.
INFO:azureml.rag.update_acs:Using Index fields: {
  "content": "content",
  "url": "url",
  "filename": "filepath",
  "title": "title",
  "metadata": "meta_json_string",
  "embedding": "contentVector"
}
INFO:azureml.rag.update_acs:Ensuring search index product-in

Successfully created index at ../data/product-info-ais-index-mlindex


### Consume the local index

In [42]:
from promptflow.rag import get_langchain_retriever_from_index

# Get the OpenAI embedded Index
retriever=get_langchain_retriever_from_index(local_index_aoai)
#retriever=get_langchain_retriever_from_index("product-info-ais-index")
retriever.get_relevant_documents("Which tent is the most waterproof")



INFO:azureml.rag.mlindex:Get ACS credential for with credential:<class 'NoneType'>.
INFO:azureml.rag.connections:The connection 'mssecureaisearch' is a <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureAISearchConnection'> with api_key auth type.
INFO:azureml.rag.connections:The connection 'mssecureai4034688619_aoai' is a <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureOpenAIConnection'> with api_key auth type.
INFO:azureml.rag.connections:The connection 'mssecureai4034688619_aoai' is a <class 'azure.ai.ml.entities._workspace.connections.connection_subtypes.AzureOpenAIConnection'> with api_key auth type.
INFO:azureml.rag.embeddings.openai:Attempt 0 to embed 1 documents.
/home/krbock/miniconda3/envs/pf-eval/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke

[Document(page_content='# Information about product item_number: 1\n\n5. **Resolution Options**:\n   - Upon receipt of the warranty claim, our customer support team will assess the issue and determine the appropriate resolution.\n   - Options may include repair, replacement of the defective parts, or, if necessary, replacement of the entire tent.\n\n6. **Limitations and Exclusions**:\n   - Our warranty is non-transferable and applies only to the original purchaser of the TrailMaster X4 Tent.\n   - The warranty does not cover any incidental or consequential damages resulting from the use or inability to use the tent.\n   - Any unauthorized repairs or alterations void the warranty.\n\n### Contact Information\n\nIf you have any questions or need further assistance, please contact our customer support:\n\n- Customer Support Phone: +1-800-123-4567\n- Customer Support Email: support@example.com\n\n## Return Policy\n- **If Membership status "None        ":**\tReturns are accepted within 30 da

In [43]:
from azure.ai.ml.entities import Index
# register the index so that it shows up in the project
cloud_index = client.indexes.create_or_update(Index(name=ais_index_name, path=local_index_aoai))

print(f"Created index '{cloud_index.name}'")
print(f"Cloud Path: {cloud_index.path}")

Method indexes: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class Index: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Created index 'product-info-ais-index'
Cloud Path: azureml://subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourcegroups/secure-ai-rg/workspaces/krbock-0635/datastores/workspaceblobstore/paths/LocalUpload/409b87928c3c49dbbc1daf85bfac8699/product-info-ais-index-mlindex


### Use a Flow to evaluate

| Evaluator | question | answer | context | ground_truth |
| --- | --- | --- | --- | --- | 
| GroundednessEvaluator | N/A | Required: String | Required: String | N/A |
| RelevanceEvaluator | Required: String | Required: String | Required: String | N/A |
| CoherenceEvaluator | Required: String | Required: String | N/A | N/A |
| FluencyEvaluator | Required: String | Required: String | N/A | N/A |
| SimilarityEvaluator | Required: String | Required: String | N/A | Required: String |
| F1ScoreEvaluator | N/A | Required: String | N/A | Required: String |
| ViolenceEvaluator | Required: String | Required: String | N/A | N/A |
| SexualEvaluator | Required: String | Required: String | N/A | N/A |
| SelfHarmEvaluator | Required: String | Required: String | N/A | N/A |
| HateUnfairnessEvaluator | Required: String | Required: String | N/A | N/A |

In [4]:
import os
from promptflow.entities import AzureOpenAIConnection, CognitiveSearchConnection

model_connect = AzureOpenAIConnection(
    name=os.environ.get("AISTUDIO_AOAI_CONNECTION_NAME"),
    api_base=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_type="azure",
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    #api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

ais_connect = CognitiveSearchConnection(
    name=os.environ.get("AISTUDIO_AIS_CONNECTION_NAME"),
    api_base=os.environ.get("AZURE_AI_SEARCH_ENDPOINT"),
    api_key=os.environ.get("AZURE_AI_SEARCH_KEY"),
)

In [1]:
myflow = "../3-metaprompt-grounding/prompt-flow/product-chat"
myflow = "../3-metaprompt-grounding/copilot-flow"

In [9]:
from promptflow.client import load_flow


flow_path = myflow
sample_input = '../data/evaluation_dataset.jsonl', # data to be evaluated

f = load_flow(source=os.path.join(myflow, "chat.prompty"))

f.context.connections = {"DetermineIntent": {"connection": model_connect}, 
                         "RetrieveDocuments": {"searchConnection": ais_connect, "embeddingModelConnection": model_connect}, 
                         "DetermineReply": {"connection": model_connect}}

result = f(url=sample_input)

print(result)

TypeError: PromptyModelConfiguration.__init__() missing 1 required positional argument: 'parameters'

In [10]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential does not work
    credential = InteractiveBrowserCredential()

In [12]:
from promptflow.azure import PFClient

# Connect to the workspace
pf = PFClient.from_config(credential=credential)

Found the config file in: /home/krbock/GitHub/rai-genai-workshop/config.json


#### Base run

In [ ]:
flow = "../3-metaprompt-grounding/copilot-flow"  # path to the flow directory
data = "./data.jsonl"  # path to the data file

# create run with the flow and data
base_run = pf.run(
    flow=flow,
    init={
        "model_config": config,
    },
    data=data,
    column_mapping={
        "question": "${data.question}",
        "chat_history": "${data.chat_history}",
    },
    stream=True,
)

#### Evaluation run

In [ ]:
eval_flow = "../3-metaprompt-grounding/copilot-flow"
config = AzureOpenAIModelConfiguration(
    connection="open_ai_connection", azure_deployment="gpt-4o"
)
eval_run = pf.run(
    flow=eval_flow,
    init={
        "model_config": config,
    },
    data="./data.jsonl",  # path to the data file
    run=base_run,  # specify base_run as the run you want to evaluate
    column_mapping={
        "answer": "${run.outputs.output}",
        "statements": "${data.statements}",
    },
    stream=True,
)

In [13]:
## Test using a flow
result = evaluate( 
    evaluation_name="qa-eval-with-flow", #name your evaluation to view in AI Studio
    target=llm_tool, # pass in a flow that you want to run then evaluate results on 
    data='../data/evaluation_dataset.jsonl', # data to be evaluated
    task_type="qa", # for different task types, different metrics are available
    metrics_list=["gpt_groundedness", "gpt_relevance", "gpt_coherence", "gpt_fluency", "gpt_similarity"], #optional superset over default set of metrics
    # model_config= { #for AI-assisted metrics, need to hook up AOAI GPT model for doing the measurement
    #         "api_version": "2023-05-15",
    #         "api_base": os.getenv("AZURE_OPENAI_ENDPOINT"),
    #         "api_type": "azure",
    #         "api_key": os.getenv("AZURE_OPENAI_KEY"),
    #         "deployment_id": os.getenv("AZURE_OPENAI_EVALUATION_DEPLOYMENT")
    # },
    data_mapping={
        "questions":"question", #column of data providing input to model
        "contexts":"context", #column of data providing context for each input
        "y_pred":"answer", #column of data providing output from model
        "y_test":"groundtruth" #column of data providing ground truth answer, optional for default metrics
        },
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI studio project
    azure_ai_project = azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and studio URL
    output_path=output_dir
)


NameError: name 'evaluate' is not defined

In [ ]:
result = evaluate(
    target=copilot_wrapper,
    evaluation_name="qa-eval-with-flow", #name your evaluation to view in AI Studio
    data='../data/evaluation_dataset.jsonl', # data to be evaluated
    evaluators={
        "relevance": relevance_eval,
        "groundedness": groundedness_eval,
        "coherence": coherence_eval,
        "fluency": fluency_eval,
        "similarity": similarity_eval,
        "f1score": f1score_eval
    },
    evaluator_config={
        "relevance": {"question": "${data.question}"},
        "coherence": {"question": "${data.question}"},
        "groundedness": {"question": "${data.question}"},
        "fluency": {"question": "${data.answer}"},
        
                "default": {
            "questions": "${data.question)", #column of data providing input to model
            #"contexts": "${data.context}", #column of data providing context for each input
            "answer": "${target.answer}", #column of data providing output from model
            "ground_truth":"${data.truth}" #column of data providing ground truth answer, optional for default metrics
        }
    },
    # to log evaluation to the cloud AI Studio project
    azure_ai_project={
        "subscription_id": os.getenv("AZURE_SUBSCRIPTION_ID"),
        "resource_group_name": os.getenv("AZURE_RESOURCE_GROUP"),
        "project_name": os.getenv("AZUREAI_PROJECT_NAME"),
    },
)

In [17]:
## Test using a flow
result = evaluate( 
    evaluation_name="qa-eval-with-flow", #name your evaluation to view in AI Studio
    target=myflow, # pass in a flow that you want to run then evaluate results on 
    data=mydata, # data to be evaluated
    task_type="qa", # for different task types, different metrics are available
    metrics_list=["gpt_groundedness", "gpt_relevance", "gpt_coherence", "gpt_fluency", "gpt_similarity"], #optional superset over default set of metrics
    model_config= { #for AI-assisted metrics, need to hook up AOAI GPT model for doing the measurement
            "api_version": "2023-05-15",
            "api_base": os.getenv("AZURE_OPENAI_ENDPOINT"),
            "api_type": "azure",
            "api_key": os.getenv("AZURE_OPENAI_KEY"),
            "deployment_id": os.getenv("AZURE_OPENAI_EVALUATION_DEPLOYMENT")
    },
    data_mapping={
        "questions":"question", #column of data providing input to model
        "contexts":"context", #column of data providing context for each input
        "y_pred":"answer", #column of data providing output from model
        "y_test":"groundtruth" #column of data providing ground truth answer, optional for default metrics
        },
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI studio project
    azure_ai_project = azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and studio URL
    output_path=output_dir
)


ValueError: target must be a callable function.

### Generate ground truth

In [1]:
#pip install azure-ai-generative[simulator]
from azure.ai.generative.synthetic.simulator import Simulator

ModuleNotFoundError: No module named 'azure.ai.generative'

In [2]:
from promptflow.evals.synthetic import AdversarialSimulator

In [ ]:
async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,
) -> dict:
    query = messages["messages"][0]["content"]
    context = None

    # Add file contents for summarization or re-write
    if 'file_content' in messages["template_parameters"]:
        query += messages["template_parameters"]['file_content']
    
    # Call your own endpoint and pass your query as input. Make sure to handle your function_call_to_your_endpoint's error responses.
    response = await function_call_to_your_endpoint(query) 
    
    # Format responses in OpenAI message protocol
    formatted_response = {
        "content": response,
        "role": "assistant",
        "context": {},
    }

    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state
    }

In [ ]:
from promptflow.evals.synthetic import AdversarialScenario

scenario = AdversarialScenario.ADVERSARIAL_QA
simulator = AdversarialSimulator(azure_ai_project=azure_ai_project)

outputs = await simulator(
        scenario=scenario, # required adversarial scenario to simulate
        target=callback, # callback function to simulate against
        max_conversation_turns=1, #optional, applicable only to conversation scenario
        max_simulation_results=3, #optional
        jailbreak=False #optional
    )

# By default simulator outputs json, use the following helper function to convert to QA pairs in jsonl format
print(outputs.to_eval_qa_json_lines())

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.resources.client import AIClient
from azure.ai.resources.entities import AzureOpenAIModelConfiguration

# initialize ai_client. This assums that config.json downloaded from ai workspace is present in the working directory
ai_client = AIClient.from_config(DefaultAzureCredential())
# Retrieve default aoai connection if it exists
aoai_connection = ai_client.get_default_aoai_connection()
# alternatively, retrieve connection by name
# aoai_connection = ai_client.connections.get("<name of connection>")

# # Specify model and deployment name for your system large language model
# aoai_config = AzureOpenAIModelConfiguration.from_connection(
#     connection=aoai_connection,
#     model_name=os.getenv('AZURE_OPENAI_EVALUATION_MODEL'),
#     deployment_name=os.getenv('AZURE_OPENAI_EVALUATION_DEPLOYMENT'),
#     temperature=0.1,
#     max_tokens=300
# )
# # Specify model and deployment name for your system large language model
aoai_config = AzureOpenAIModelConfiguration.from_connection(
    connection=aoai_connection,
    model_name='gpt-4-32k',
    deployment_name='gpt-4-32k-0613',
    temperature=0.1,
    max_tokens=300
)

In [ ]:
import os

In [ ]:
from openai import AsyncAzureOpenAI
oai_client = AsyncAzureOpenAI(api_key=os.getenv('AZURE_OPENAI_KEY'), azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'), api_version="2024-02-15-preview")
async_oai_chat_completion_fn = oai_client.chat.completions.create

In [ ]:
function_simulator = Simulator.from_fn(
    fn=async_oai_chat_completion_fn, # Simulate against a local function OR callback function
    simulator_connection=aoai_config # Configure the simulator
) 

In [ ]:
template = Simulator.get_template("summarization")

In [ ]:
template_params = [
    {
        "name": "John Doe",
        "chatbot_name": "AI Chatbot",
        "filename": "company_report.txt",
        "file_content": "The company is doing well. The stock price is up 10% this quarter. The company is expanding into new markets. The company is investing in new technology. The company is hiring new employees. The company is launching new products. The company is opening new stores. The company is increasing its market share. The company is increasing its revenue. The company is increasing its profits.",
    },
    {
        "name": "Jane Doe",
        "chatbot_name": "AI Chatbot",
        "filename": "sales_report.txt",
        "file_content": "The sales team is doing well. The sales team is meeting its targets. The sales team is increasing its revenue. The sales team is increasing its market share. The sales team is increasing its profits. The sales team is expanding into new markets. The sales team is launching new products. The sales team is opening new stores. The sales team is hiring new employees. The sales team is investing in new technology.",
    },
]

In [ ]:
outputs = await function_simulator.simulate_async(
    template,
    parameters=template_params,
    max_conversation_turns=2,
    api_call_delay_sec=10,
    max_simulation_results=10,
)

### Generate QA from files

In [ ]:
from pathlib import Path
# product sample data
texts_glob = Path("../data/product-info/")
# azureai-samples data
#texts_glob = Path("../../azureai-samples/scenarios/generate-synthetic-data/ai-generated-data-qna/data/data_generator_texts/")
files = Path.glob(texts_glob, pattern="**/*")
files = [file for file in files if Path.is_file(file)]

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import NLTKTextSplitter
import nltk

# download pre-trained Punkt tokenizer for sentence splitting
nltk.download("punkt")

text_splitter = NLTKTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",  # encoding for gpt-4 and gpt-35-turbo
    chunk_size=300,  # number of tokens to split on
    chunk_overlap=0,
)
texts = []
for file in files:
    loader = UnstructuredFileLoader(file)
    docs = loader.load()
    data = docs[0].page_content
    texts += text_splitter.split_text(data)
print(f"Number of texts after splitting: {len(texts)}")

In [ ]:
from azure.ai.generative.synthetic.qa import QADataGenerator, QAType

## Uses AzureOpenAI environment variables

# For granular logs you may set DEBUG log level:
import logging
#logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(level=logging.ERROR)

model_config = {
    "deployment": "gpt-4-1106-preview",
    "model": "gpt-4",
    "max_tokens": 2000,
}

qa_generator = QADataGenerator(model_config=model_config)

In [ ]:
QADataGenerator(model_config=model_config)

#### Generate QA asynchronously

In [ ]:
from azure.ai.generative.synthetic.qa import QADataGenerator, QAType
import asyncio
from collections import Counter
from typing import Dict

concurrency = 3  # number of concurrent calls
sem = asyncio.Semaphore(concurrency)

qa_type = QAType.CONVERSATION


async def generate_async(text: str) -> Dict:
    async with sem:
        return await qa_generator.generate_async(
            text=text,
            qa_type=qa_type,
            num_questions=3,  # Number of questions to generate per text
        )


results = await asyncio.gather(*[generate_async(text) for text in texts], return_exceptions=True)

question_answer_list = []
token_usage = Counter()
for result in results:
    if isinstance(result, Exception):
        raise result  # exception raised inside generate_async()
    question_answer_list.append(result["question_answers"])
    token_usage += result["token_usage"]

print("Successfully generated QAs")

In [ ]:
print(f"Tokens used: {result['token_usage']}")

In [ ]:
question_answer_list

### Save the generated data for later use
Let us save the generated QnA in a format which can be understood by prompt flow (for evaluation, batch runs). 

In [ ]:
import os
generated_dir = "../data/generated_qa"
os.makedirs(generated_dir, exist_ok=True)
output_file = os.path.join(generated_dir, "generated_qa.jsonl")
qa_generator.export_to_file(output_file, qa_type, question_answer_list)